In [26]:
import numpy as np
import itertools
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [18]:
class Agent():
    def __init__(self, allocations = None, n_actions = 13, n_soldiers = 100, strict = True):
        if allocations is None:
            self.allocations = np.random.multinomial(n_soldiers, np.ones(n_actions) / 13)
        else:
            if (sum(allocations) != n_soldiers) and strict:
                raise Exception("Only allocated {} of {} soldiers, {} remaining}".format(sum(allocations), n_soldiers, n))
            self.allocations = np.array(allocations)
        self.wins = 0
        self.reset()
    def reset(self):
        self.wins = 0
    def add_win(self):
        self.wins += 1
    def reproduce(self, n = 1, include_self = False):
        offspring = [Agent(allocations = np.random.multinomial(n_soldiers, (self.allocations + 1) / (n_soldiers + n_actions))) for _ in range(n)]
        if include_self:
            offspring[0] = Agent(allocations = self.allocations)
        return offspring
    
    def __repr__(self):
        return str((self.allocations, self.wins))

In [3]:
def run_round(agents : list):
    agent_actions = len(agents[0].allocations)
    for agent in agents:
        assert(len(agent.allocations == agent_actions))
        agent.reset()
    for (a, b) in itertools.combinations(agents, 2):
        a_score = 0
        b_score = 0
        for i in range(agent_actions):
            if a.allocations[i] > b.allocations[i]:
                a_score += i + 1
            elif a.allocations[i] < b.allocations[i]:
                b_score += i + 1
            else:
                a_score += (i + 1.0) / 2.0
                b_score += (i + 1.0) / 2.0
        if a_score > b_score:
            a.add_win()
        else:
            b.add_win()

In [4]:
def selection_reproduction(agents, reproduce_denom = 2, replace = False):
    assert(round(len(agents) / reproduce_denom) == len(agents) / float(reproduce_denom))
    total_wins = sum([agent.wins for agent in agents])
    reproducing_indices = np.random.choice(len(agents), int(len(agents) / reproduce_denom), 
                                          p = [agent.wins / total_wins for agent in agents], replace = replace)
    children = []
    for i in reproducing_indices:
        children += agents[i].reproduce(reproduce_denom, include_self = True)
    assert(len(children) == len(agents))
    return children

In [5]:
def run_game(n_rounds = 100, n_agents = 100, replace = False):
    agents = [Agent() for _ in range(n_agents)]
    for i in range(n_rounds):
        run_round(agents)
        agents = selection_reproduction(agents, reproduce_denom = 5, replace = replace)
        clear_output(wait = True)
        print("Finished round {} of {}".format(i, n_rounds))
    run_round(agents)
    return agents

In [6]:
def sorted_by_wins(agents):
    return sorted(agents, key = lambda x: x.wins, reverse = True)

In [251]:
winning_agents1 = sorted_by_wins(run_game(n_rounds = 200, n_agents = 200, replace = True))

Finished round 199 of 200


In [252]:
winning_agents2 = sorted_by_wins(run_game(n_rounds = 200, n_agents = 200, replace = False))

Finished round 199 of 200


In [262]:
winning_agents3 = sorted_by_wins(run_game(n_rounds= 100, n_agents = 100, replace = True))
winning_agents4 = sorted_by_wins(run_game(n_rounds = 100, n_agents = 100, replace = False))

Finished round 99 of 100


In [15]:
sum(range(14))

91

In [196]:
                               #1, 2, 3, 4, 5, 6, 7,   8,  9, 10,  11,  12, 13
my_agent = Agent(allocations = [0, 0, 3, 1, 6, 6, 1, 15, 16, 21,  21,  9,  1])

In [267]:
                                  #1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13
# other_agent = Agent(allocations = [1,  2,  0,  3,  7,  8,  9, 13,  1, 13, 12, 13, 18])
# other_agent = Agent(allocations = [1, 2, 4, 5, 0, 8, 9, 13, 2, 13, 12, 13, 18])

In [255]:
run_round(winning_agents + [my_agent])

NameError: name 'winning_agents' is not defined

In [263]:
all_winning_agents = winning_agents1 + winning_agents2 + winning_agents3 + winning_agents4

In [268]:
run_round(all_winning_agents + [other_agent])
other_agent

(array([ 1,  2,  4,  5,  0,  8,  9, 13,  2, 13, 12, 13, 18]), 479)

In [269]:
sorted_by_wins(all_winning_agents)[:20]

[(array([ 3,  2,  3,  2,  3,  5,  3,  6, 10, 11, 21, 13, 18]), 489),
 (array([ 1,  1,  1,  6,  7,  7, 12,  5,  1, 10, 12, 17, 20]), 488),
 (array([ 5,  1,  0,  5,  8,  4,  8,  4,  0, 12, 19, 13, 21]), 488),
 (array([ 4,  1,  1,  5,  7,  6, 12,  4,  0, 10, 18, 13, 19]), 478),
 (array([ 4,  1,  3,  2,  3,  6,  5,  8, 11,  8, 16, 19, 14]), 477),
 (array([ 2,  2,  2,  4,  3,  4, 11,  5,  5,  8, 15, 15, 24]), 472),
 (array([ 1,  2,  0,  3,  7, 10, 13,  6,  0,  7, 18, 17, 16]), 469),
 (array([ 5,  0,  4,  3,  1,  5,  5,  7,  7, 13, 15, 20, 15]), 468),
 (array([ 2,  1,  1,  7,  8,  6,  2,  8, 15,  2, 13, 15, 20]), 466),
 (array([ 5,  4,  3,  6,  2,  6,  4,  7, 10,  7, 15, 14, 17]), 465),
 (array([ 0,  0,  5,  3,  9,  8,  7, 11,  6,  5, 14, 10, 22]), 464),
 (array([ 5,  2,  2,  6,  3,  9,  2,  6, 10,  8, 16, 18, 13]), 463),
 (array([ 2,  4,  4,  5,  2,  6,  6,  9,  2, 11, 20, 13, 16]), 462),
 (array([ 0,  4,  2,  1,  4,  4, 13,  7,  5, 14, 10, 17, 19]), 460),
 (array([ 4,  2,  0,  3,  7,  4,  

(array([ 0,  0,  3,  1,  6,  6,  1, 15, 16, 21, 21,  9,  1]), 292)

In [20]:
real_agents = [Agent(alloc, 10) for alloc in np.genfromtxt("castle-solutions-4.csv", delimiter=",", skip_header=1) 
              if sum(alloc) == 100]

In [21]:
run_round(real_agents)

In [22]:
sorted_by_wins(real_agents)

[(array([ 0.,  0.,  0., 12., 19.,  2.,  3.,  3., 35., 26.]), 784),
 (array([ 0.,  0.,  0., 13., 17.,  1.,  1.,  1., 31., 36.]), 777),
 (array([ 1.,  1.,  1., 15., 17.,  1.,  1.,  1., 30., 32.]), 777),
 (array([ 0.,  1.,  1., 15., 18.,  1.,  3.,  2., 35., 24.]), 771),
 (array([ 0.,  0., 10.,  1.,  1., 23.,  3.,  3., 34., 25.]), 769),
 (array([ 0.,  0., 12.,  1.,  1., 23.,  2.,  1., 35., 25.]), 761),
 (array([ 0.,  0.,  0., 14., 18.,  0.,  0.,  0., 34., 34.]), 760),
 (array([ 0.,  0.,  8., 11., 13., 23.,  3.,  3.,  6., 33.]), 760),
 (array([ 0.,  7.,  1.,  2.,  2.,  1., 26.,  1., 34., 26.]), 760),
 (array([ 0.,  0.,  0., 16., 21.,  1.,  1.,  1., 34., 26.]), 758),
 (array([ 0.,  7.,  0.,  0.,  1.,  0., 28.,  1., 35., 28.]), 758),
 (array([ 0.,  0., 11.,  1.,  1., 23.,  4.,  3., 33., 24.]), 757),
 (array([ 0.,  0.,  1., 12.,  1., 24.,  1., 30.,  1., 30.]), 756),
 (array([ 0.,  0., 10.,  2.,  3., 22.,  3.,  3., 32., 25.]), 756),
 (array([ 0.,  0.,  8., 13., 13., 22.,  3.,  3.,  6., 32.]), 7

In [ ]:
(winning_agents1[0], winning_agents2[0])

In [ ]:
sum(range(1, 11))

In [64]:
sum([agent.allocations[9] == 0 for agent in real_agents]) / len(real_agents)

0.12247071352502663

In [47]:
sum([True, True])

2

In [271]:
sum(range(14)) / 2

45.5